In [1]:
import pandas as pd
import sqlite3 as db
from yfapi import YahooFinanceAPI, Interval
import datetime
import utils

In [2]:
con = db.connect('database.db')

In [3]:
cn = con.cursor()

In [4]:
coins= ['BTC', 'ETH', 'BNB', 'XRP', 'DOGE', 'LTC', 'WETH', 'SOL', 'SHIB', 'MATIC']

In [5]:
now = pd.to_datetime('2022-10-25').date()

In [6]:
print(now)

2022-11-25


In [7]:
coins_df  = pd.DataFrame()

In [8]:
coins_df['coin'] = coins

In [9]:
coins_df['last_predicted'] = now

In [10]:
coins_df['last_trained'] = now

In [11]:
coins_df

,coin,last_predicted,last_trained
0,BTC,2022-11-25,2022-11-25
1,ETH,2022-11-25,2022-11-25
2,BNB,2022-11-25,2022-11-25
3,XRP,2022-11-25,2022-11-25
4,DOGE,2022-11-25,2022-11-25
5,LTC,2022-11-25,2022-11-25
6,WETH,2022-11-25,2022-11-25
7,SOL,2022-11-25,2022-11-25
8,SHIB,2022-11-25,2022-11-25
9,MATIC,2022-11-25,2022-11-25


In [12]:
coins_df.to_sql('coins', con, if_exists='replace')

10

In [13]:
dh = YahooFinanceAPI(Interval.DAILY)
then = datetime.datetime(2017, 1, 1)

In [14]:
df = dh.get_ticker_data("BTC-USD", then, now)

In [22]:
df.isna().sum().sum()

0

In [16]:
df = utils.preprocess_data(df)

In [18]:
df

,Open,High,Low,Close,Volume
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,344945984
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008
...,...,...,...,...,...
2022-11-21,16291.223633,16291.223633,15599.046875,15787.284180,37429485518
2022-11-22,15782.300781,16253.047852,15656.606445,16189.769531,30726828760
2022-11-23,16195.588867,16638.193359,16170.502930,16610.707031,32958875628
2022-11-24,16611.636719,16771.474609,16501.767578,16604.464844,26129037414


In [23]:
for coin in coins:
    s = coin + '-USD'
    df = dh.get_ticker_data(s, then, now)
    if df.isna().sum().sum() == 0:
        df = utils.preprocess_data(df)
        df.to_sql(coin, con, if_exists='replace')
    else:
        print(coin + ' has null')
        

WETH has null


In [24]:
weth = dh.get_ticker_data("WETH-USD", then, now)

In [26]:
weth.isna().sum()

Date         0
Open         1
High         1
Low          1
Close        1
Adj Close    1
Volume       1
dtype: int64

In [33]:
weth.isna()[weth.isna()['Open']==True]

,Date,Open,High,Low,Close,Adj Close,Volume
220,False,True,True,True,True,True,True


In [37]:
weth.iloc[220]

Date         2018-08-22 00:00:00
Open                         NaN
High                         NaN
Low                          NaN
Close                        NaN
Adj Close                    NaN
Volume                       NaN
Name: 220, dtype: object

In [41]:
cn.execute('PRAGMA table_list')

In [ ]:
print(cn.fetchall())

[('main', 'SHIB', 'table', 6, 0, 0), ('main', 'sqlite_schema', 'table', 5, 0, 0), ('main', 'coins', 'table', 4, 0, 0), ('main', 'BTC', 'table', 6, 0, 0), ('main', 'ETH', 'table', 6, 0, 0), ('main', 'MATIC', 'table', 6, 0, 0), ('main', 'XRP', 'table', 6, 0, 0), ('main', 'DOGE', 'table', 6, 0, 0), ('main', 'LTC', 'table', 6, 0, 0), ('main', 'BNB', 'table', 6, 0, 0), ('main', 'SOL', 'table', 6, 0, 0), ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]
